In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, LSTM, concatenate, Dropout
from tensorflow.keras.utils import plot_model
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint

# seed
import os
seed = 123
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.set_random_seed(seed)

In [2]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [3]:
image_generator = ImageDataGenerator(width_shift_range=0.1,
                                     height_shift_range=0.1, 
                                     zoom_range=[0.8,1.2],
                                     #brightness_range=[0.8,1.2], 
                                     shear_range=10)

In [4]:
x1 = train.drop(['id', 'digit', 'letter'], axis=1).values
x1 = x1.reshape(-1, 28, 28, 1)
x1 = x1/255
x1_remake = []
for i in range(x1.shape[0]):
    num_aug = 0
    tmp = x1[i]
    tmp = tmp.reshape((1,) + tmp.shape)
    for x_aug in image_generator.flow(tmp, batch_size = 1) :
        if num_aug >= 1:
            break
        x1_remake.append(x_aug[0])
        num_aug += 1
x1_remake = np.array(x1_remake)

x1_total = np.concatenate((x1, x1_remake), axis=0)
print(x1_total.shape)

(4096, 28, 28, 1)


In [5]:
y1_data = train['digit']
y1 = np.zeros((len(y1_data), len(y1_data.unique())))
for i, digit in enumerate(y1_data):
    y1[i, digit] = 1

y1_remake = y1.copy()
y1_total = np.concatenate((y1, y1_remake), axis=0)
print(y1_total.shape)

(4096, 10)


In [6]:
x1_let = train['letter'].values
x1_let = x1_let[:, np.newaxis]
en = OneHotEncoder()
x1_let = en.fit_transform(x1_let).toarray()

x1_remake_let = x1_let.copy()

x1_letter_total = np.concatenate((x1_let, x1_remake_let), axis=0)
x1_letter_total.shape

(4096, 26)

In [7]:
x1_train, x1_val, y1_train, y1_val = train_test_split(x1_total, y1_total, test_size=0.2, shuffle=True, stratify=y1_total)

print(x1_train.shape)
print(x1_val.shape)
print(y1_train.shape)
print(y1_val.shape)

x1_letter_train = x1_letter_total[:x1_train.shape[0],:]
x1_letter_val = x1_letter_total[x1_train.shape[0]:,:]
print(x1_letter_train.shape)
print(x1_letter_val.shape)

(3276, 28, 28, 1)
(820, 28, 28, 1)
(3276, 10)
(820, 10)
(3276, 26)
(820, 26)


In [8]:
x2_train = np.reshape(x1_train, (x1_train.shape[0], x1_train.shape[1], x1_train.shape[2]))
x2_val = np.reshape(x1_val, (x1_val.shape[0], x1_val.shape[1], x1_val.shape[2]))
y2_train = y1_train.copy()
y2_val = y1_val.copy()
x2_letter_train = x1_letter_train.copy()
x2_letter_val = x1_letter_val.copy()

print(x2_train.shape)
print(x2_val.shape)
print(y2_train.shape)
print(y2_val.shape)
print(x2_letter_train.shape)
print(x2_letter_val.shape)

(3276, 28, 28)
(820, 28, 28)
(3276, 10)
(820, 10)
(3276, 26)
(820, 26)


In [9]:
x3_train = np.reshape(x1_train, (x1_train.shape[0], x1_train.shape[1]*x1_train.shape[2]))
x3_val = np.reshape(x1_val, (x1_val.shape[0], x1_val.shape[1]*x1_val.shape[2]))
y3_train = y1_train.copy()
y3_val = y1_val.copy()
x3_letter_train = x1_letter_train.copy()
x3_letter_val = x1_letter_val.copy()

print(x3_train.shape)
print(x3_val.shape)
print(y3_train.shape)
print(y3_val.shape)
print(x3_letter_train.shape)
print(x3_letter_val.shape)

(3276, 784)
(820, 784)
(3276, 10)
(820, 10)
(3276, 26)
(820, 26)


In [10]:
input1 = Input(shape=(28,28,1))
x1 = Conv2D(64, (2,2), activation='relu', padding='same')(input1)
x1 = Dropout(0.2)(x1)
x1 = MaxPooling2D((2,2))(x1)
x1 = Conv2D(64, (2,2), activation='relu', padding='same')(x1)
x1 = Dropout(0.2)(x1)
x1 = MaxPooling2D((2,2))(x1)
x1 = Conv2D(128, (2,2), activation='relu', padding='same')(x1)
x1 = Dropout(0.2)(x1)
x1 = MaxPooling2D((2,2))(x1)
x1 = Flatten()(x1)

input2 = Input(shape=(26,))
merge1 = concatenate([x1, input2])
x2 = Dense(500, activation='relu')(merge1)
x2 = Dropout(0.2)(x2)
x2 = Dense(100, activation='relu')(x2)

input3 = Input(shape=(28,28))
x3 = LSTM(512, activation='relu', return_sequences=True)(input3)
x3 = Dropout(0.2)(x3)
x3 = LSTM(256, activation='relu', return_sequences=True)(x3)
x3 = Dropout(0.2)(x3)
x3 = LSTM(128, activation='relu')(x3)
x3 = Dropout(0.2)(x3)
x3 = Dense(100, activation='relu')(x3)

input4 = Input(shape=(26,))
merge2 = concatenate([x3, input4])
x4 = Dense(100, activation='relu')(merge2)

input5 = Input(shape=(784,))
x5 = Dense(1000, activation='relu')(input5)
x5 = Dropout(0.2)(x5)
x5 = Dense(500, activation='relu')(x5)
x5 = Dropout(0.2)(x5)
x5 = Dense(100, activation='relu')(x5)

input6 = Input(shape=(26,))
merge3 = concatenate([x5, input6])
x6 = Dense(100, activation='relu')(merge3)

merge = concatenate([x2, x4, x6])

x7 = Dense(200, activation='relu')(merge)
x7 = Dropout(0.2)(x7)

x8 = Dense(50, activation='relu')(x7)
x8 = Dropout(0.2)(x8)
output1 = Dense(10, activation='softmax')(x8)

x9 = Dense(50, activation='relu')(x7)
x9 = Dropout(0.2)(x9)
output2 = Dense(10, activation='softmax')(x9)

x10 = Dense(50, activation='relu')(x7)
x10 = Dropout(0.2)(x10)
output3 = Dense(10, activation='softmax')(x10)

model = Model(inputs = [input1, input2, input3, input4, input5, input6], outputs = [output1, output2, output3])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 28, 28, 1)    0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 28, 28, 64)   320         input_1[0][0]                    
__________________________________________________________________________________________________
dropout (Dropout)               (None, 28, 28, 64)   0           conv2d[0][0]                     
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 14, 14, 64)   0           dropout[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (

In [11]:
model.compile(optimizer = 'adam', metrics = ['accuracy'], loss = 'categorical_crossentropy', 
              loss_weights = [0.8, 0.1, 0.1])
history = model.fit([x1_train, x1_letter_train, x2_train, x2_letter_train, x3_train, x3_letter_train], [y1_train, y2_train, y3_train],
                    validation_data=([x1_val, x1_letter_val, x2_val, x2_letter_val, x3_val, x3_letter_val], [y1_val, y2_val, y3_val]), 
                    batch_size=64, epochs=50, verbose=1, 
                    callbacks = [ModelCheckpoint('./models/{epoch:02d}-{val_dense_10_acc:.4f}.h5',
                    monitor='val_dense_10_acc', verbose=1, save_best_only=True, mode='auto')])

Train on 3276 samples, validate on 820 samples
Epoch 1/50
3276/3276 [==============================] - 24s 7ms/step - loss: 2.2994 - dense_10_loss: 2.2972 - dense_12_loss: 2.3051 - dense_14_loss: 2.3109 - dense_10_acc: 0.1230 - dense_12_acc: 0.1123 - dense_14_acc: 0.1013 - val_loss: 2.2682 - val_dense_10_loss: 2.2662 - val_dense_12_loss: 2.2762 - val_dense_14_loss: 2.2758 - val_dense_10_acc: 0.1488 - val_dense_12_acc: 0.1293 - val_dense_14_acc: 0.1573

Epoch 00001: val_dense_10_acc improved from -inf to 0.14878, saving model to ./models/01-0.1488.h5
Epoch 2/50
3276/3276 [==============================] - 15s 5ms/step - loss: 2.2029 - dense_10_loss: 2.1949 - dense_12_loss: 2.2326 - dense_14_loss: 2.2367 - dense_10_acc: 0.1725 - dense_12_acc: 0.1566 - dense_14_acc: 0.1587 - val_loss: 2.1198 - val_dense_10_loss: 2.1119 - val_dense_12_loss: 2.1536 - val_dense_14_loss: 2.1487 - val_dense_10_acc: 0.2207 - val_dense_12_acc: 0.2049 - val_dense_14_acc: 0.2341

Epoch 00002: val_dense_10_acc impr

3276/3276 [==============================] - 15s 5ms/step - loss: 0.2930 - dense_10_loss: 0.2906 - dense_12_loss: 0.2995 - dense_14_loss: 0.3056 - dense_10_acc: 0.9023 - dense_12_acc: 0.9005 - dense_14_acc: 0.8962 - val_loss: 1.2387 - val_dense_10_loss: 1.2417 - val_dense_12_loss: 1.2242 - val_dense_14_loss: 1.2291 - val_dense_10_acc: 0.6244 - val_dense_12_acc: 0.6232 - val_dense_14_acc: 0.6293

Epoch 00017: val_dense_10_acc improved from 0.61951 to 0.62439, saving model to ./models/17-0.6244.h5
Epoch 18/50
3276/3276 [==============================] - 16s 5ms/step - loss: 0.2617 - dense_10_loss: 0.2587 - dense_12_loss: 0.2752 - dense_14_loss: 0.2716 - dense_10_acc: 0.9148 - dense_12_acc: 0.9139 - dense_14_acc: 0.9127 - val_loss: 1.4251 - val_dense_10_loss: 1.4331 - val_dense_12_loss: 1.4034 - val_dense_14_loss: 1.3828 - val_dense_10_acc: 0.5939 - val_dense_12_acc: 0.5915 - val_dense_14_acc: 0.6012

Epoch 00018: val_dense_10_acc did not improve from 0.62439
Epoch 19/50
3276/3276 [======

3276/3276 [==============================] - 16s 5ms/step - loss: 0.0948 - dense_10_loss: 0.0924 - dense_12_loss: 0.1054 - dense_14_loss: 0.1036 - dense_10_acc: 0.9744 - dense_12_acc: 0.9689 - dense_14_acc: 0.9707 - val_loss: 1.4376 - val_dense_10_loss: 1.4393 - val_dense_12_loss: 1.4426 - val_dense_14_loss: 1.4183 - val_dense_10_acc: 0.6476 - val_dense_12_acc: 0.6463 - val_dense_14_acc: 0.6500

Epoch 00034: val_dense_10_acc did not improve from 0.65976
Epoch 35/50
3276/3276 [==============================] - 15s 5ms/step - loss: 0.0558 - dense_10_loss: 0.0551 - dense_12_loss: 0.0601 - dense_14_loss: 0.0569 - dense_10_acc: 0.9850 - dense_12_acc: 0.9844 - dense_14_acc: 0.9829 - val_loss: 1.5072 - val_dense_10_loss: 1.5108 - val_dense_12_loss: 1.5009 - val_dense_14_loss: 1.4849 - val_dense_10_acc: 0.6488 - val_dense_12_acc: 0.6439 - val_dense_14_acc: 0.6537

Epoch 00035: val_dense_10_acc did not improve from 0.65976
Epoch 36/50
3276/3276 [==============================] - 15s 5ms/step - 

In [12]:
#from tensorflow.keras.models import load_model
#best_model = load_model('./models/46-0.7463.h5')

In [13]:
#x1_test = test.drop(['id', 'letter'], axis=1).values
#x1_test = x1_test.reshape(-1, 28, 28, 1)
#x1_test = x1_test/255

#x2_test = test.drop(['id', 'letter'], axis=1).values
#x2_test = x2_test.reshape(-1, 28, 28)
#x2_test = x2_test/255

#x1_letter_test = test['letter'].values
#x1_letter_test = x1_letter_test[:, np.newaxis]
#en = OneHotEncoder()
#x1_letter_test = en.fit_transform(x1_letter_test).toarray()

#x2_letter_test = x1_letter_test.copy()

#y1_test, y2_test = best_model.predict([x1_test, x1_letter_test, x2_test, x2_letter_test])
#y_1 = np.argmax(y1_test, axis=1)
#y_2 = np.argmax(y2_test, axis=1)
#print(y_1)
#print(y_2)

[6 5 2 ... 6 9 0]
[6 0 2 ... 6 9 0]


In [14]:
#submission = pd.read_csv('data/submission.csv')
#submission['digit'] = y_1

In [15]:
#submission.to_csv('data/submission(val7463).csv', index=False)

In [16]:
#submission['digit'] = y_2
#submission.to_csv('data/submission2.csv', index=False)